In [1]:
from io import StringIO
import time

import requests
import pandas as pd


In [2]:
class APIException(Exception):
    pass

class API:
    def __init__(self, owner, title, user, pw, base_url="http://127.0.0.1:8000"):
        self.owner = owner
        self.title = title
        self.auth = (user, pw)
        self.base_url = f"{base_url}/{owner}/{title}/api/v1"
        
    def create(self, adjustment: dict = None, meta_parameters: dict = None):
        adjustment = adjustment or {}
        meta_parameters = meta_parameters or {}
        resp = requests.post(
            f"{self.base_url}/",
            json={
                "model_parameters": adjustment, 
                "meta_parameters": meta_parameters
            },
            auth=self.auth
        )
        if resp.status_code == 201:
            return resp.json()
        raise APIException(resp.text)
    
    def detail(self, model_pk):
        while True:
            resp = requests.get(f"{self.base_url}/{model_pk}")
            if resp.status_code == 202:
                pass
            elif resp.status_code == 200:
                return resp.json()
            else:
                raise APIException(resp.text)
            time.sleep(1)
    
    def results(self, model_pk):
        result = self.detail(model_pk)
        res = {}
        for output in result["outputs"]["downloadable"]:
            if output["media_type"] == "CSV":
                res[output["title"]] = pd.read_csv(
                    StringIO(output["data"])
                )
            else:
                print(f'{output["media_type"]} not implemented yet')
        return res
    

In [3]:
api = API("hdoupe", "Matchups", "hdoupe", "heyhey2222")

In [4]:
res = api.create(
    adjustment={
        "matchup": {
            "pitcher": "Max Scherzer"
        }
    }
)

In [5]:
res

{'inputs': {'meta_parameters': {'use_full_data': True},
  'model_parameters': {'matchup': {'pitcher': 'Max Scherzer'}},
  'inputs_file': None,
  'errors_warnings': {'matchup': {'errors': {}, 'warnings': {}},
   'GUI': {'errors': {}, 'warnings': {}},
   'API': {'errors': {}, 'warnings': {}}}},
 'outputs': None,
 'traceback': None,
 'creation_date': '2019-05-21T17:11:38.321621-05:00',
 'api_url': '/hdoupe/Matchups/api/v1/83',
 'gui_url': '/hdoupe/Matchups/83/',
 'eta': 9.91,
 'model_pk': 83}

In [6]:
result = api.results(res["model_pk"])

In [7]:
result.keys()

dict_keys(['Max Scherzer v. All batters', 'Max Scherzer v. Chipper Jones', 'Max Scherzer v. David Ortiz'])

In [8]:
all_df = result["Max Scherzer v. All batters"]
all_df.loc[all_df.pitch_type == "FF", "release_speed"].mean()

94.21551601423516